In [1]:
from src.dataloader.datapipes import VideoReader
from tqdm import tqdm
import pathlib

In [2]:
paths = list(map(str, list(pathlib.Path('/Volumes/External/ssv2/data').glob('**/*.webm'))))[0:100]

In [3]:
for i, _ in enumerate(VideoReader(paths, n_workers=8, queue_size=32)):
    pass
i

No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 150 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 86 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 23 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 59 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 5 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 21 more times)
No accelerated colorspace conversion found from yuv4

99

In [6]:
i.dtype

dtype('uint8')

In [ ]:
import av

container = av.open(next(paths))

In [ ]:
import av
import io


def new_output(output_path, width, height):
    output_container = av.open(output_path, 'w', 'mp4')
    output_stream = output_container.add_stream('h264', rate=30, options={'preset': 'veryslow', 'tune': 'fastdecode'})
    output_stream.pix_fmt = 'yuv420p'
    output_stream.width = width
    output_stream.height = height
    return output_container, output_stream


def calc_new_dims(width, height, short_edge):
    if width < height:
        new_width = short_edge
        new_height = int(height * short_edge / width)
    else:
        new_height = short_edge
        new_width = int(width * short_edge / height)
    return new_width, new_height


def resize_video(input_path, output_path, short_edge=224):
    input_container = av.open(input_path)
    output_container, output_stream = None, None

    for frame in input_container.decode(video=0):
        w, h = calc_new_dims(frame.width, frame.height, short_edge)
        
        if output_container is None:
            output_container, output_stream = new_output(output_path, w, h)
            
        frame = frame.reformat(width=w, height=h, format='rgb24', interpolation=av.video.reformatter.Interpolation.LANCZOS)
        for packet in output_stream.encode(frame):
            output_container.mux(packet)

    for packet in output_stream.encode():
        output_container.mux(packet)
    
    input_container.close()
    output_container.close()

In [ ]:
file = io.BytesIO()
resize_video(next(paths), file, 224)